In [ ]:
Name = 'Parsa Ghezelbash'
Student_ID = '401110437'

In [ ]:
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Part 1

$$
\min_{x} \quad \frac{1}{2} \|Ax - b\|_2^2 + \gamma \|x\|_1
$$

- \( $A \in \mathbb{R}^{m \times n}$ \) is the input matrix.
- \( $b \in \mathbb{R}^{m}$ \) is the target vector.
- \( $\gamma > 0$ \) is the regularization parameter that controls sparsity.

### Proximal Gradient Descent
$$
\text{prox}_{\lambda}(y) = \text{sign}(y) \cdot \max(|y| - \lambda, 0)
$$

The algorithm iterates until convergence or until a maximum number of iterations is reached. The step size is chosen based on the Lipschitz constant of \( $A^TA$ \).

### ADMM for LASSO
$$
\min_{x, y} \quad \frac{1}{2} \|Ax - b\|_2^2 + \gamma \|y\|_1 \quad \text{subject to} \quad x = y
$$

The augmented Lagrangian for this problem is given by:

$$
L(x, y, \mu) = \frac{1}{2} \|Ax - b\|_2^2 + \gamma \|y\|_1 + \mu^T (x - y) + \frac{\rho}{2} \|x - y\|_2^2
$$

where:
- \( $\mu$ \) is the dual variable (Lagrange multiplier).
- \( $\rho > 0$ \) is the penalty parameter.

The ADMM algorithm consists of the following updates at each iteration:
1. **\( x \)-update:** Solve the quadratic subproblem:
   $$
   x^{k+1} = \arg\min_{x} \left( \frac{1}{2} \|Ax - b\|_2^2 + \frac{\rho}{2} \|x - y^k + \frac{\mu^k}{\rho}\|_2^2 \right)
   $$
2. **\( y \)-update:** Apply the soft-thresholding operator:
   $$
   y^{k+1} = \text{prox}_{\frac{\gamma}{\rho}}\left(x^{k+1} + \frac{\mu^k}{\rho}\right)
   $$
3. **\( \mu \)-update:** Update the dual variable:
   $$
   \mu^{k+1} = \mu^k + \rho (x^{k+1} - y^{k+1})
   $$

The iterations continue until convergence, which is typically determined by checking the primal and dual residuals:
$$
\text{Primal residual:} \quad r^k = \|x^{k} - y^{k}\|_2
$$
$$
\text{Dual residual:} \quad s^k = \rho \|y^{k} - y^{k-1}\|_2
$$


In [14]:
def solve_with_cvxpy(m, n, gamma=0.1):
    np.random.seed(0)
    A = np.random.randn(m, n)
    b = np.random.randn(m)
    A = A / np.linalg.norm(A, ord=2)
    b = b / np.linalg.norm(b, ord=2)

    start_time = time.time()
    x = cp.Variable(n)
    loss = (1/2) * cp.norm2(A @ x - b)**2 + gamma * cp.norm1(x)
    problem = cp.Problem(cp.Minimize(loss))
    problem.solve()
    cvxpy_time = time.time() - start_time

    print(f"CVXPY solution time (m={m}, n={n}): {cvxpy_time:.4f} seconds")
    print(f"CVXPY optimal value (m={m}, n={n}): {problem.value:.4f}")
    return problem.value

def soft_thresholding(y, lambd):
    return np.sign(y) * np.maximum(np.abs(y) - lambd, 0)

def proximal_gradient(A, b, gamma, stepsize, max_iters=1000, tol=1e-6):
    n = A.shape[1]
    x = np.zeros(n)
    L = np.linalg.norm(A.T @ A, ord=2)
    stepsize = min(stepsize, 1 / (2 * L))
    history = $]
    for k in range(max_iters):
        grad = A.T @ (A @ x - b)
        x_new = soft_thresholding(x - stepsize * grad, stepsize * gamma)
        obj_value = 0.5 * np.linalg.norm(A @ x_new - b)**2 + gamma * np.linalg.norm(x_new, 1)
        history.append(obj_value)
        if np.any(np.isnan(x_new)) or np.any(np.isinf(x_new)):
            print("Numerical instability detected. Stopping iteration.")
            break
        if np.linalg.norm(x_new - x, ord=2) < tol:
            break
        x = x_new
    return x, history

def evaluate_proximal_gradient(m, n, gamma=0.1, stepsizes=[0.01, 0.05, 0.1]):
    np.random.seed(0)
    A = np.random.randn(m, n)
    b = np.random.randn(m)
    A = A / np.linalg.norm(A, ord=2)
    b = b / np.linalg.norm(b, ord=2)

    for stepsize in stepsizes:
        start_time = time.time()
        x_pg, history = proximal_gradient(A, b, gamma, stepsize)
        pg_time = time.time() - start_time
        print(f"Proximal Gradient solution time (m={m}, n={n}, stepsize={stepsize}): {pg_time:.4f} seconds")
        print(f"Proximal Gradient optimal value (m={m}, n={n}, stepsize={stepsize}): {history[-1]:.4f}")

def admm_lasso(A, b, gamma, rho, max_iters=1000, tol=1e-6):
    m, n = A.shape
    x = np.zeros(n)
    y = np.zeros(n)
    mu = np.zeros(n)
    ATA = A.T @ A
    ATb = A.T @ b
    inv_matrix = la.inv(rho * np.eye(n) + ATA)
    history = []

    for k in range(max_iters):
        x = inv_matrix @ (rho * y - mu + ATb)
        y = soft_thresholding(x + mu / rho, gamma / rho)
        mu += rho * (x - y)
        obj_value = 0.5 * np.linalg.norm(A @ x - b)**2 + gamma * np.linalg.norm(x, 1)
        history.append(obj_value)
        if np.any(np.isnan(x)) or np.any(np.isinf(x)):
            print("Numerical instability detected. Stopping iteration.")
            break
        if np.linalg.norm(x - y, ord=2) < tol:
            break
    return x, history

def evaluate_admm(m, n, gamma=0.1, rhos=[0.1, 1, 10]):
    np.random.seed(0)
    A = np.random.randn(m, n)
    b = np.random.randn(m)
    A = A / np.linalg.norm(A, ord=2)
    b = b / np.linalg.norm(b, ord=2)

    for rho in rhos:
        start_time = time.time()
        x_admm, history = admm_lasso(A, b, gamma, rho)
        admm_time = time.time() - start_time
        print(f"ADMM solution time (m={m}, n={n}, rho={rho}): {admm_time:.4f} seconds")
        print(f"ADMM optimal value (m={m}, n={n}, rho={rho}): {history[-1]:.4f}")

for m, n in [(100, 50), (200, 100), (300, 150)]:
    print(f"\nEvaluating CVXPY for m={m}, n={n}")
    solve_with_cvxpy(m, n)

    print(f"\nEvaluating Proximal Gradient for m={m}, n={n}")
    evaluate_proximal_gradient(m, n)

    print(f"\nEvaluating ADMM for m={m}, n={n}")
    evaluate_admm(m, n)


Evaluating CVXPY for m=100, n=50
CVXPY solution time (m=100, n=50): 0.0114 seconds
CVXPY optimal value (m=100, n=50): 0.4833

Evaluating Proximal Gradient for m=100, n=50
Proximal Gradient solution time (m=100, n=50, stepsize=0.01): 0.0175 seconds
Proximal Gradient optimal value (m=100, n=50, stepsize=0.01): 0.4833
Proximal Gradient solution time (m=100, n=50, stepsize=0.05): 0.0088 seconds
Proximal Gradient optimal value (m=100, n=50, stepsize=0.05): 0.4833
Proximal Gradient solution time (m=100, n=50, stepsize=0.1): 0.0045 seconds
Proximal Gradient optimal value (m=100, n=50, stepsize=0.1): 0.4833

Evaluating ADMM for m=100, n=50
ADMM solution time (m=100, n=50, rho=0.1): 0.0017 seconds
ADMM optimal value (m=100, n=50, rho=0.1): 0.4833
ADMM solution time (m=100, n=50, rho=1): 0.0007 seconds
ADMM optimal value (m=100, n=50, rho=1): 0.4833
ADMM solution time (m=100, n=50, rho=10): 0.0026 seconds
ADMM optimal value (m=100, n=50, rho=10): 0.4833

Evaluating CVXPY for m=200, n=100
CVXPY 

# Part 2

### CVXPY and ADMM for Graph Learning
In this part of the script, the graph learning problem is solved using CVXPY and ADMM. The objective is to minimize the following function:

$$
\min_{w \geq 0} \quad 2 z^T w - \alpha \sum \log(Qw + 1e-6) + \beta \|w\|_2^2
$$

where:
- \( $z$ \) is a vector derived from the upper triangular part of a symmetric matrix \( $Z$ \).
- \( $Q$ \) is a random matrix.
- \( $\alpha, \beta > 0$ \) are regularization parameters.

**CVXPY Solver:** The problem is formulated in CVXPY, and the solver directly minimizes the objective subject to the constraint \( $w \geq 0$ \).

**ADMM Solver:** The ADMM algorithm for this problem involves:
1. **\( $w$ \)-update:**
   $$
   w^{k+1} = \max\left(w^k - \tau_1 \rho Q^T(Qw^k - v^k - \frac{\lambda^k}{\rho}) - \frac{2 \tau_1 z}{2 \tau_1 \beta + 1}, 0\right)
   $$
2. **\( $v$ \)-update:**
   $$
   v^{k+1} = \frac{1}{2}\left((1 - \tau_2 \rho)v^k + \tau_2 \rho Qw^{k+1} - \tau_2 \lambda^k + \sqrt{((1 - \tau_2 \rho)v^k + \tau_2 \rho Qw^{k+1} - \tau_2 \lambda^k)^2 + 4 \alpha \tau_2}\right)
   $$
3. **Dual variable update:**
   $$
   \lambda^{k+1} = \lambda^k + \rho (Qw^{k+1} - v^{k+1})
   $$

The iterations continue until the primal and dual residuals fall below a specified tolerance.

In [ ]:
def extract_upper_triangular(Z):
    return Z[np.triu_indices_from(Z, k=1)]

def solve_with_cvxpy(N, alpha=0.1, beta=0.01):
    np.random.seed(0)
    m = int(N * (N - 1) / 2)
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2
    z = extract_upper_triangular(Z)
    z = z / np.linalg.norm(z, ord=2)

    Q = np.random.rand(N, m)
    Q = Q / np.linalg.norm(Q, ord=2)

    w = cp.Variable(m)
    loss = 2 * z.T @ w - alpha * cp.sum(cp.log(Q @ w + 1e-6)) + beta * cp.norm(w, 2)**2
    constraints = [w >= 0]
    problem = cp.Problem(cp.Minimize(loss), constraints)

    start_time = time.time()
    problem.solve()
    cvxpy_time = time.time() - start_time

    print(f"CVXPY solution time (N={N}): {cvxpy_time:.4f} seconds")
    print(f"CVXPY optimal value (N={N}): {problem.value:.4f}")
    return problem.value

def admm_graph_learning(z, Q, alpha, beta, rho, max_iters=1000, tol=1e-6):
    m = len(z)
    s = Q.shape[0]
    w = np.zeros(m)
    v = np.ones(s)
    lam = np.zeros(s)

    tau1 = 1 / (2 * (Q.shape[0] - 1) * rho)
    tau2 = 1 / rho

    for k in range(max_iters):
        w_tilde = w - tau1 * rho * Q.T @ (Q @ w - v - lam / rho)
        w_new = np.maximum(w_tilde - 2 * tau1 * z / (2 * tau1 * beta + 1), 0)

        v_tilde = (1 - tau2 * rho) * v + tau2 * rho * Q @ w_new - tau2 * lam
        v_tilde = np.clip(v_tilde, a_min=-1e3, a_max=1e3)
        v_tilde_squared = v_tilde**2
        v_new = (v_tilde + np.sqrt(v_tilde_squared + 4 * alpha * tau2)) / 2

        lam += rho * (Q @ w_new - v_new)

        rp = np.linalg.norm(Q @ w_new - v_new, ord=2)
        rd = np.linalg.norm(Q.T @ (v_new - v), ord=2)

        if rp < tol and rd < tol:
            break

        if np.any(np.isnan(w_new)) or np.any(np.isinf(w_new)) or np.any(np.isnan(v_new)) or np.any(np.isinf(v_new)):
            print("Numerical instability detected. Stopping iteration.")
            break

        w, v = w_new, v_new

    return w


def evaluate_admm(N, alpha=0.1, beta=0.01, rho=1.0, max_iters=1000, tol=1e-6):
    np.random.seed(0)
    m = int(N * (N - 1) / 2)
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2
    z = extract_upper_triangular(Z)
    z = z / np.linalg.norm(z, ord=2)

    Q = np.random.rand(N, m)
    Q = Q / np.linalg.norm(Q, ord=2)

    start_time = time.time()
    w_admm = admm_graph_learning(z, Q, alpha, beta, rho, max_iters, tol)
    admm_time = time.time() - start_time

    obj_admm = 2 * z.T @ w_admm - alpha * np.sum(np.log(Q @ w_admm + 1e-6)) + beta * np.linalg.norm(w_admm, 2)**2

    print(f"ADMM solution time (N={N}): {admm_time:.4f} seconds")
    print(f"ADMM optimal value (N={N}): {obj_admm:.4f}")
    return obj_admm


for N in [30, 100]:
    print(f"\nEvaluating CVXPY for N={N}")
    solve_with_cvxpy(N)

    print(f"\nEvaluating ADMM for N={N}")
    evaluate_admm(N, rho=1.0, max_iters=2000, tol=1e-6)



Evaluating CVXPY for N=30
CVXPY solution time (N=30): 0.0163 seconds
CVXPY optimal value (N=30): -62.1359

Evaluating ADMM for N=30
ADMM solution time (N=30): 0.2651 seconds
ADMM optimal value (N=30): 41.4465

Evaluating CVXPY for N=100
CVXPY solution time (N=100): 0.8912 seconds
CVXPY optimal value (N=100): -66.8370

Evaluating ADMM for N=100
ADMM solution time (N=100): 3.5950 seconds
ADMM optimal value (N=100): 138.1551
